<a href="https://colab.research.google.com/github/prajwalraju/GoogleColab/blob/main/Wav2Lip_TenDeepfake_eng.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# **Get the code and models**

In [9]:
#@title <h1>Install Wav2Lip</h1>
#@markdown * Install dependencies
#@markdown * Download models
!rm -rf /content/sample_data
!mkdir /content/sample_data

!git clone https://github.com/zabique/Wav2Lip

#download the pretrained model
!wget 'https://iiitaphyd-my.sharepoint.com/personal/radrabha_m_research_iiit_ac_in/_layouts/15/download.aspx?share=EdjI7bZlgApMqsVoEUUXpLsBxqXbn5z8VTmoxp55YNDcIA' -O '/content/Wav2Lip/checkpoints/wav2lip_gan.pth'
!wget 'https://iiitaphyd-my.sharepoint.com/:u:/g/personal/radrabha_m_research_iiit_ac_in/Eb3LEzbfuKlJiR600lQWRxgBIY27JZg80f7V9jtMfbNDaQ?e=TBFBVW' -O '/content/Wav2Lip/checkpoints/wav2lip.pth'
!pip install https://raw.githubusercontent.com/AwaleSajil/ghc/master/ghc-1.0-py3-none-any.whl

# !pip uninstall tensorflow tensorflow-gpu
!cd Wav2Lip && pip install -r requirements.txt

#download pretrained model for face detection
!wget "https://www.adrianbulat.com/downloads/python-fan/s3fd-619a316812.pth" -O "/content/Wav2Lip/face_detection/detection/sfd/s3fd.pth"

!pip install -q youtube-dl
!pip install ffmpeg-python
!pip install librosa==0.8.1

from google.colab import drive
drive.mount('/content/drive')


Done




---



# **Now lets try!**



---



In [13]:
#@title 1.Create Wav2Lip video (using wav2lip_gan.pth) GAN
!cd Wav2Lip && python inference.py --checkpoint_path checkpoints/wav2lip_gan.pth --face "/content/sample_data/input_vidoee.mp4" --audio "/content/drive/MyDrive/input_audio.wav" --outfile /content/out_pad.mp4

Using cuda for inference.
Reading video frames...
Number of frames available for inference: 180
(80, 264)
Length of mel chunks: 95
  0% 0/1 [00:00<?, ?it/s]
  0% 0/6 [00:00<?, ?it/s]
 17% 1/6 [00:09<00:45,  9.18s/it]
 33% 2/6 [00:11<00:20,  5.07s/it]
 50% 3/6 [00:13<00:11,  3.76s/it]
 67% 4/6 [00:15<00:06,  3.15s/it]
 83% 5/6 [00:17<00:02,  2.81s/it]
100% 6/6 [00:25<00:00,  4.24s/it]
Load checkpoint from: checkpoints/wav2lip_gan.pth
  0% 0/1 [00:27<?, ?it/s]
Traceback (most recent call last):
  File "/content/Wav2Lip/inference.py", line 280, in <module>
    main()
  File "/content/Wav2Lip/inference.py", line 252, in main
    model = load_model(args.checkpoint_path)
  File "/content/Wav2Lip/inference.py", line 171, in load_model
    checkpoint = _load(path)
  File "/content/Wav2Lip/inference.py", line 162, in _load
    checkpoint = torch.load(checkpoint_path)
  File "/usr/local/lib/python3.9/dist-packages/torch/serialization.py", line 795, in load
    return _legacy_load(opened_file, ma

In [6]:
#@title 3.Play result video -  50% scaling
from IPython.display import HTML
from base64 import b64encode
mp4 = open('/content/out.mp4','rb').read()
data_url = "data:video/mp4;base64," + b64encode(mp4).decode()
HTML(f"""
<video width="50%" height="50%" controls>
      <source src="{data_url}" type="video/mp4">
</video>""")

In [14]:
#@title 4.Download Result.mp4 to your computer
from google.colab import files
files.download('/content/out_pad.mp4') 


<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>